code from Feb 28 
currently having issues with subprocess check_output returning non-zero exit status when creating the html


###  How to open a file dialog box
Tkinter is the easiest way if you don't want to have any 
other dependencies. 
An alternative that uses tkinter is 
https://easygui.readthedocs.io/en/master/


```python
# open file dialog box
import tkinter as tk
from tkinter import filedialog

# To show only the dialog without any 
# other GUI elements, you have to hide the root window 
# using the withdraw method:
root = tk.Tk()
root.withdraw()

print("Select the Jupyter Notebook you wish to convert")
fn = filedialog.askopenfilename()
print(file_path)
```

### How to run a shell command from python
```python
# run a shell command from python and get the response
# as a string into a varaiable which you can print or discard
from subprocess import check_output
response = check_output("dir C:", shell=True)
print (response)
```

#### How to convert a Jupyter Notebook to HTML using nbconvert
##### verify that you have nbconvert installed as part of Anaconda
```cmd
jupyter nbconvert --help
```

The simplest way to use nbconvert is
    > jupyter nbconvert mynotebook.ipynb
which will convert mynotebook.ipynb to the default format (probably HTML).

You can specify the export format with `--to`.
Options include ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'rst', 'script', 'slides']

In [1]:
import os
# open file dialog box
import tkinter as tk
from tkinter import filedialog
import subprocess
# from subprocess import check_output

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.lift()
root.withdraw()
input("Anything")

print("Select the Jupyter Notebook you wish to convert")
fn = filedialog.askopenfilename()
# print(file_path)
# jupyter nbconvert --to html --template basic Jupyter_Book_Notes.ipynb
print (fn)
cmd = "jupyter nbconvert --to html --template basic {0}".format(fn)
print (cmd)

os.system(cmd)

# try:
    # response = check_output(cmd, shell=True)
    # response = check_output(['jupyter','nbconvert --to html --template basic ' + fn])
    # print(response)
# except subprocess.CalledProcessError as e:
#    print (e)
    


Anythingf
Select the Jupyter Notebook you wish to convert

jupyter nbconvert --to html --template basic 


-1

In [20]:
# loop through a collection of notebooks and md files
import os
import tkinter as tk
from tkinter import filedialog

# set properties for the directory open dialog box
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.lift()
root.withdraw()

# for OSX
# self.root.lift()
# self.root.call('wm', 'attributes', '.', '-topmost', True)
# self.root.after_idle(self.root.call, 'wm', 'attributes', '.', '-topmost', False)


print("Select the Directory containing Jupyter Notebooks you wish to convert")
path = filedialog.askdirectory()


for root, directories, filenames in os.walk(path):
    #for directory in directories:
    for filename in filenames:
        infile = os.path.join(root,filename)
        outfile = os.path.join(root,filename)
        outfile += ".html"
        if filename.endswith("ipynb"):    
            cmd = "jupyter nbconvert --output \"{0}\" --to html --template basic \"{1}\"".format(outfile,infile)
            print (cmd)
            os.system(cmd) 
        if filename.endswith("md"):
            cmd = "grip \"{0}\" --export \"{1}\"".format(infile,outfile)
            print (cmd)
            os.system(cmd)             
            
print ("done!")            
          

Select the Directory containing Jupyter Notebooks you wish to convert
jupyter nbconvert --output "C:/Temp/aaa/My Book\Chapter 1\Section 1\Custom Display Logic.ipynb.html" --to html --template basic "C:/Temp/aaa/My Book\Chapter 1\Section 1\Custom Display Logic.ipynb"
jupyter nbconvert --output "C:/Temp/aaa/My Book\Chapter 1\Section 1\Running Code.ipynb.html" --to html --template basic "C:/Temp/aaa/My Book\Chapter 1\Section 1\Running Code.ipynb"
jupyter nbconvert --output "C:/Temp/aaa/My Book\Chapter 1\Section 2\programming_environment_osx.ipynb.html" --to html --template basic "C:/Temp/aaa/My Book\Chapter 1\Section 2\programming_environment_osx.ipynb"
jupyter nbconvert --output "C:/Temp/aaa/My Book\Chapter 1\Section 2\Python+Fundamentals.ipynb.html" --to html --template basic "C:/Temp/aaa/My Book\Chapter 1\Section 2\Python+Fundamentals.ipynb"
jupyter nbconvert --output "C:/Temp/aaa/My Book\Chapter 2\Section 1\Nbconvert Support.ipynb.html" --to html --template basic "C:/Temp/aaa/My Book\

### Random notes

must install python grip... add notes explaining this

https://nbconvert.readthedocs.io/en/latest/index.html

https://www.blog.pythonlibrary.org/2018/10/09/how-to-export-jupyter-notebooks-into-other-formats/